In [1]:
import platform
print("python version " + platform.python_version())
import sys
import json
import re
import os
from os.path import exists
import logging

In [2]:
title = "Create Parameter Set for Point-Based Model"
name = "create_parameter_set_for_point_based_model"

root_params = [
    ["Latitude","float",1,"Latitude position for simulation"],
    ["Longitude","float",1,"Longitude position for simulation"],
    ["MetSet","dropdown:opt1,opt2","opt1","MetSet data input"],
    ["Urban elemenets","checkbox",0,"Use urban elements in simulation"],
    ["Fire elemenets","checkbox",0,"Use fire elements in simulation"],
    ["Crop elemenets","checkbox",0,"Use crop elements in simulation"]
]

parameters = {
    "Urban_parameters":[
        ['CANYON_HWR','float',1,'canyon height to width ratio (unitless)'],
        ['EM_IMPROAD','float',1,'emissivity of impervious road (unitless)'],
        ['EM_PERROAD','float',1,'emissivity of pervious road (unitless)'],
        ['EM_ROOF','float',1,'emissivity of roof (unitless)'],
        ['EM_WALL','float',1,'emissivity of wall (unitless)'],
        ['HT_ROOF','float',1,'height of roof (meters)'],
        ['THICK_ROOF','float',1,'thickness of roof (meters)'],
        ['THICK_WALL','float',1,'thickness of wall (meters)'],
        ['T_BUILDING_MAX','float',1,'maximum interior building temperature (K)'],
        ['T_BUILDING_MIN','float',1,'minimum interior building temperature (K)'],
        ['WIND_HGT_CANYON','float',1,'height of wind in canyon (meters)'],
        ['WTLUNIT_ROOF','float',1,'fraction of roof (unitless)'],
        ['WTROAD_PERV','float',1,'fraction of pervious road (unitless)'],
        ['ALB_IMPROAD_DIR','float',1,'direct albedo of impervious road (unitless)'],
        ['ALB_IMPROAD_DIF','float',1,'diffuse albedo of impervious road (unitless)'],
        ['ALB_PERROAD_DIR','float',1,'direct albedo of pervious road (unitless)'],
        ['ALB_PERROAD_DIF','float',1,'diffuse albedo of pervious road (unitless)'],
        ['ALB_ROOF_DIR','float',1,'direct albedo of roof (unitless)'],
        ['ALB_ROOF_DIF','float',1,'diffuse albedo of roof (unitless)'],
        ['ALB_WALL_DIR','float',1,'direct albedo of wall (unitless)'],
        ['ALB_WALL_DIF','float',1,'diffuse albedo of wall (unitless)'],
        ['TK_ROOF','float',1,'thermal conductivity of roof (W/m*K)'],
        ['TK_WALL','float',1,'thermal conductivity of wall (W/m*K)'],
        ['TK_IMPROAD','float',1,'thermal conductivity of impervious road (W/m*K)'],
        ['CV_ROOF','float',1,'volumetric heat capacity of roof (J/m^3*K)'],
        ['CV_WALL','float',1,'volumetric heat capacity of wall (J/m^3*K)'],
        ['CV_IMPROAD','float',1,'volumetric heat capacity of impervious road (J/m^3*K)'],
        ['NLEV_IMPROAD','int',1,'number of impervious road layers (unitless)']
    ],
    "Fire_parameters":[
        ['CANYON_HWR2','float',1,'canyon height to width ratio (unitless)'],
        ['EM_IMPROAD2','float',1,'emissivity of impervious road (unitless)']
    ],
    "Crop_parameters":[
        ['CANYON_HWR3','float',1,'canyon height to width ratio (unitless)'],
        ['EM_IMPROAD3','float',1,'emissivity of impervious road (unitless)']
    ],
    "Land_properties":[
        ['mxsoil_color','int',1,'maximum numbers of soil colors (unitless)'],
        ['mxsoil_order','int',1,'maximum numbers of soil order (unitless)'],
        ['SOIL_COLOR','int',1,'soil color (unitless)'],
        ['SOIL_ORDER','int',1,'soil order (unitless)'],
        ['PCT_SAND','float',1,'percent sand (unitless)'],
        ['PCT_CLAY','float',1,'percent clay (unitless)'],
        ['ORGANIC','float',1,'organic matter density at soil levels (kg/m3 (assumed carbon content 0.58 gC per gOM))'],
        ['FMAX','float',1,'maximum fractional saturated area (unitless)'],
        ['natpft','int',1,'indices of natural PFTs (index)'],
        ['LANDFRAC_PFT','float',1,'land fraction from pft dataset (unitless)'],
        ['PFTDATA_MASK','int',1,'land mask from pft dataset, indicative of real/fake points (unitless)'],
        ['PCT_NATVEG','float',1,'total percent natural vegetation landunit (unitless)'],
        ['PCT_CROP','float',1,'total percent crop landunit (unitless)']
    ],
    "Plant_composition":[
        ['Plant_type','dropdown:not_vegetated,needleleaf_evergreen_temperate_tree,needleleaf_evergreen_boreal_tree,needleleaf_deciduous_boreal_tree,broadleaf_evergreen_tropical_tree,broadleaf_evergreen_temperate_tree,broadleaf_deciduous_tropical_tree,broadleaf_deciduous_temperate_tree,broadleaf_deciduous_boreal_tree,broadleaf_evergreen_shrub,broadleaf_deciduous_temperate_shrub,broadleaf_deciduous_boreal_shrub,c3_arctic_grass,c3_non-arctic_grass,c4_grass,c3_crop,c3_irrigated,corn,irrigated_corn,spring_temperate_cereal,irrigated_spring_temperate_cereal,winter_temperate_cereal,irrigated_winter_temperate_cereal,soybean,irrigated_soybean',"not_vegetated",'Type of plant associated with specified fraction composition'],
        ['Fraction','float',0.1,'Fraction of plants of the specified type']
    ],
    "Plant_leaf_parameters":[
        ['Leaf_type','dropdown:needleleaf_evergreen_temperate_tree,needleleaf_evergreen_boreal_tree,needleleaf_deciduous_boreal_tree,broadleaf_evergreen_tropical_tree,broadleaf_evergreen_temperate_tree,broadleaf_deciduous_tropical_tree,broadleaf_deciduous_temperate_tree,broadleaf_deciduous_boreal_tree,broadleaf_evergreen_shrub,broadleaf_deciduous_temperate_shrub,broadleaf_deciduous_boreal_shrub,c3_arctic_grass,c3_non-arctic_grass,c4_grass,c3_crop,c3_irrigated,corn,irrigated_corn,spring_temperate_cereal,irrigated_spring_temperate_cereal,winter_temperate_cereal,irrigated_winter_temperate_cereal,soybean,irrigated_soybean',"not_vegetated",'Type of plant associated with specified fraction composition'],
        ['aleaff','float',1,'unitless (Leaf Allocation coefficient parameter used in CNAllocationn)'],
        ['leafcn_obs','float',1,'leaf CN ratio (gC/gN)'],
        ['allconsl','float',1,'unitless (Leaf Allocation coefficient parameter power used in CNAllocation)'],
        ['bfact','float',1,'unitless (Exponential factor used in CNAllocation for fraction allocated to leaf)'],
        ['fleafcn','float',1,'gC/gN (Leaf C:N during organ fill)'],
        ['fleafi','float',1,'unitless (Leaf Allocation coefficient parameter fraction used in CNAllocation)'],
        ['leaf_long','float',1,'years (Leaf longevity)']
    ],
    "Plant_root_parameters":[
        ['Root_type','dropdown:needleleaf_evergreen_temperate_tree,needleleaf_evergreen_boreal_tree,needleleaf_deciduous_boreal_tree,broadleaf_evergreen_tropical_tree,broadleaf_evergreen_temperate_tree,broadleaf_deciduous_tropical_tree,broadleaf_deciduous_temperate_tree,broadleaf_deciduous_boreal_tree,broadleaf_evergreen_shrub,broadleaf_deciduous_temperate_shrub,broadleaf_deciduous_boreal_shrub,c3_arctic_grass,c3_non-arctic_grass,c4_grass,c3_crop,c3_irrigated,corn,irrigated_corn,spring_temperate_cereal,irrigated_spring_temperate_cereal,winter_temperate_cereal,irrigated_winter_temperate_cereal,soybean,irrigated_soybean',"not_vegetated",'Type of plant associated with specified fraction composition']
    ],
    "Plant_biomass_allocation":[
        ['Biomass_type','dropdown:not_vegetated,needleleaf_evergreen_temperate_tree,needleleaf_evergreen_boreal_tree,needleleaf_deciduous_boreal_tree,broadleaf_evergreen_tropical_tree,broadleaf_evergreen_temperate_tree,broadleaf_deciduous_tropical_tree,broadleaf_deciduous_temperate_tree,broadleaf_deciduous_boreal_tree,broadleaf_evergreen_shrub,broadleaf_deciduous_temperate_shrub,broadleaf_deciduous_boreal_shrub,c3_arctic_grass,c3_non-arctic_grass,c4_grass,c3_crop,c3_irrigated,corn,irrigated_corn,spring_temperate_cereal,irrigated_spring_temperate_cereal,winter_temperate_cereal,irrigated_winter_temperate_cereal,soybean,irrigated_soybean',"not_vegetated",'Type of plant associated with specified fraction composition']
    ],
    "Dynamic_plant_composition":[
        ['Composition_type','dropdown:not_vegetated,needleleaf_evergreen_temperate_tree,needleleaf_evergreen_boreal_tree,needleleaf_deciduous_boreal_tree,broadleaf_evergreen_tropical_tree,broadleaf_evergreen_temperate_tree,broadleaf_deciduous_tropical_tree,broadleaf_deciduous_temperate_tree,broadleaf_deciduous_boreal_tree,broadleaf_evergreen_shrub,broadleaf_deciduous_temperate_shrub,broadleaf_deciduous_boreal_shrub,c3_arctic_grass,c3_non-arctic_grass,c4_grass,c3_crop,c3_irrigated,corn,irrigated_corn,spring_temperate_cereal,irrigated_spring_temperate_cereal,winter_temperate_cereal,irrigated_winter_temperate_cereal,soybean,irrigated_soybean',"not_vegetated",'Type of plant associated with specified fraction composition'],
        ['Dynamic_fraction','float',0.1,'Fraction of plants of the specified type and time'],
        ['Time','float',0,'Time of specified composition']     
    ]
}

Order = [
    "ROOT:Longitude",
    "ROOT:Latitude",
    "ROOT:MetSet",
    "ROOT:Urban elements",
    "Urban parameters",
    "ROOT:Fire elements",
    "Fire parameters",
    "ROOT:Crop elements",
    "Crop parameters",
    "Land properties",
    "Plant composition",
    "Plant leaf parameters",
    "Plant root parameters",
    "Plant biomass allocation",
    "Dynamic plant composition"
]


In [3]:
yaml_params = {}
yaml_groups = {}
spec = {
    "name": name,
    "ver": "0.0.1",
    "authors": [
        "pweisenhorn"
    ],
    "contact": "http://kbase.us/contact-us/",
    "visible": True,
    "categories": [
        "active"
    ],
    "widgets": {
        "output": "no-display",
        "input": None
    },
    "parameter-groups": [],
    "parameters":[],
    "behavior": {
        "service-mapping": {
            "url": "",
            "name": "Test",
            "method": name,
            "input_mapping": [
                {
                    "target_property": "workspace",
                    "narrative_system_variable": "workspace"
                }
            ],
            "output_mapping": [
                {
                    "service_method_output_path": [
                        0,
                        "report_name"
                    ],
                    "target_property": "report_name"
                },
                {
                    "service_method_output_path": [
                        0,
                        "report_ref"
                    ],
                    "target_property": "report_ref"
                },
                {
                    "narrative_system_variable": "workspace",
                    "target_property": "workspace"
                }
            ]
        }
    },
    "job_id_output_field": "docker"
}

for item in root_params:
    spec["behavior"]["service-mapping"]["input_mapping"].append({
        "target_property": item[0],
        "input_parameter": item[0]
    })
    newparam = {
        "id" : item[0],
        "allow_multiple" : False,
        "optional" : False,
        "advanced" : False,
        "default_values" : [str(item[2])]
    }
    spec["parameters"].append(newparam)
    yaml_params[item[0]] = {"ui-name":item[0],"short-hint":item[0]}
    if item[1] == "float" or item[1] == "int":
        newparam["field_type"] = "text"
        newparam["text_options"] = {"validate_as":item[1]}
    elif item[1] == "checkbox":
        newparam["field_type"] = "checkbox"
        newparam["checkbox_options"] = {
            "unchecked_value" : 0,
            "checked_value" : 1
        }
    else:
        newparam["field_type"] = "dropdown"
        newparam["dropdown_options"] = {
            "options": []
        }
        items = item[1][9:].split(",")
        for option in items:
            newparam["dropdown_options"]["options"].append({
                "value": option,
                "display": option,
                "id": option,
                "ui_name": option
            })

for param_name in parameters:
    newgparam = {
        "id":param_name,
        "optional": False,
        "advanced": False,
        "allow_multiple": True,
        "with_border": True,
        "parameters":[]
    }
    spec["parameter-groups"].append(newgparam)
    yaml_groups[param_name] = {"ui-name":param_name,"short-hint":param_name}
    spec["behavior"]["service-mapping"]["input_mapping"].append({
        "target_property": param_name,
        "input_parameter": param_name
    })
    for param_tuple in parameters[param_name]:
        newgparam["parameters"].append(param_tuple[0])
        newparam = {
            "id" : param_tuple[0],
            "allow_multiple" : False,
            "optional" : False,
            "advanced" : False,
            "default_values" : [str(param_tuple[2])]
        }
        spec["parameters"].append(newparam)
        yaml_params[param_tuple[0]] = {"ui-name":param_tuple[0],"short-hint":param_tuple[0]}
        if param_tuple[1] == "float" or param_tuple[1] == "int":
            newparam["field_type"] = "text"
            newparam["text_options"] = {"validate_as":param_tuple[1]}
        elif param_tuple[1] == "checkbox":
            newparam["field_type"] = "checkbox"
            newparam["checkbox_options"] = {
                "unchecked_value" : 0,
                "checked_value" : 1
            }
        else:
            newparam["field_type"] = "dropdown"
            newparam["dropdown_options"] = {
                "options": []
            }
            items = param_tuple[1][9:].split(",")
            for option in items:
                newparam["dropdown_options"]["options"].append({
                    "value": option,
                    "display": option,
                    "id": option,
                    "ui_name": option
                })
print(json.dumps(spec,indent=4, sort_keys=False))

{
    "name": "create_parameter_set_for_point_based_model",
    "ver": "0.0.1",
    "authors": [
        "pweisenhorn"
    ],
    "contact": "http://kbase.us/contact-us/",
    "visible": true,
    "categories": [
        "active"
    ],
    "widgets": {
        "output": "no-display",
        "input": null
    },
    "parameter-groups": [
        {
            "id": "Urban_parameters",
            "optional": false,
            "advanced": false,
            "allow_multiple": true,
            "with_border": true,
            "parameters": [
                "CANYON_HWR",
                "EM_IMPROAD",
                "EM_PERROAD",
                "EM_ROOF",
                "EM_WALL",
                "HT_ROOF",
                "THICK_ROOF",
                "THICK_WALL",
                "T_BUILDING_MAX",
                "T_BUILDING_MIN",
                "WIND_HGT_CANYON",
                "WTLUNIT_ROOF",
                "WTROAD_PERV",
                "ALB_IMPROAD_DIR",
                "ALB

In [4]:
import yaml
display = {
    "name":title,
    "tooltip":title,
    "screenshots":[],
    "suggestions":{
        "apps":{
            "related":[],
            "next":[]
        },
        "methods":{
            "related":[],
            "next":[]
        }
    },
    "parameters":yaml_params,
    "parameter-groups":yaml_groups,
    "icon":"modelseed-teal.png",
    "description":"<p></p>"
}
print(yaml.dump(display, sort_keys=False))

name: Create Parameter Set for Point-Based Model
tooltip: Create Parameter Set for Point-Based Model
screenshots: []
suggestions:
  apps:
    related: []
    next: []
  methods:
    related: []
    next: []
parameters:
  Latitude:
    ui-name: Latitude
    short-hint: Latitude
  Longitude:
    ui-name: Longitude
    short-hint: Longitude
  MetSet:
    ui-name: MetSet
    short-hint: MetSet
  Urban elemenets:
    ui-name: Urban elemenets
    short-hint: Urban elemenets
  Fire elemenets:
    ui-name: Fire elemenets
    short-hint: Fire elemenets
  Crop elemenets:
    ui-name: Crop elemenets
    short-hint: Crop elemenets
  CANYON_HWR:
    ui-name: CANYON_HWR
    short-hint: CANYON_HWR
  EM_IMPROAD:
    ui-name: EM_IMPROAD
    short-hint: EM_IMPROAD
  EM_PERROAD:
    ui-name: EM_PERROAD
    short-hint: EM_PERROAD
  EM_ROOF:
    ui-name: EM_ROOF
    short-hint: EM_ROOF
  EM_WALL:
    ui-name: EM_WALL
    short-hint: EM_WALL
  HT_ROOF:
    ui-name: HT_ROOF
    short-hint: HT_ROOF
  THICK_RO